In [1]:
from dotenv import dotenv_values
import requests
import sys


"""
Configuration
"""

path_to_src_folder = "../src"
sfblic_link = "https://www.sfbli.com/"
path_to_clean_corpus = "../data/clean_content.txt"

# Load environment variables
config = dotenv_values("../.env")
google_api_key = config['GOOGLE_API_KEY']
search_engine_id = config['SEARCH_ENGINE_ID']

sys.path.append(path_to_src_folder)

from helpers.google_interactor import GoogleInteractor
from helpers.html_parser import HTMLParser 
from helpers.storage_manager import StorageManager
from helpers.refiner_model import RefinerModel
from helpers.improver_model import ImproverModel
from helpers.writer_citer_model import WriterCiterModel
from helpers.visibility import parse_citations_from_text, compute_citation_scores

"""
============================================
Interact with Helpers to mimic Perplexity
============================================
"""

"""
Set up
"""

# Number of links to consider
K = 5

# Select a query to use for the current trial
query = "What is life insurance?"

# Instantiate helpers
google_interactor = GoogleInteractor(api_key=google_api_key, search_engine_id=search_engine_id)
website_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
writer_citer = WriterCiterModel(api_key=config['OPEN_AI_API_KEY'])

# Submit the query to Google
search_results = google_interactor.search_google(query=query, num_results=K)

"""
View links
"""

top_k_links = [item['link'] for item in search_results['items']][:K]
top_k_links


['https://www.libertymutual.com/insurance-resources/life/how-does-life-insurance-work',
 'https://www.benefits.va.gov/insurance/',
 'https://content.naic.org/insurance-topics/life-insurance',
 'https://www.va.gov/life-insurance/options-eligibility/valife/',
 'https://www.opm.gov/healthcare-insurance/life-insurance/']

In [2]:
raw_text_from_html = []

# For each item of raw text that was stored ...
for link in top_k_links:    

    try:

        # Fet the HTML associated with the link
        response = requests.get(link)
    
        # Get the raw text associated with the HTML
        parsed_text = HTMLParser.parse_html(response.content)
    
        # Store the raw text from the HTML
        raw_text_from_html.append({"link" : link, "raw_text" : parsed_text})

    except:
        print("Something happened with ", link)


"""
Get the refined text from the raw text
"""


website_refiner_responses = []
# For each element of raw text that was stored ...
for current_item in raw_text_from_html:
    
    link = current_item['link']
    raw_text = current_item['raw_text']   

    
    response_from_website_refiner = website_refiner.refine_text(raw_text=raw_text, query=query)
    
    if response_from_website_refiner:
        response_from_website_refiner['link'] = link
        website_refiner_responses.append(response_from_website_refiner)
    else:
        print("Failed to refine text.")


"""
Print the refined text
"""
for item in website_refiner_responses:
    print(item['link'])
    print(item['choices'][0]['message']['content'])
    print()


https://www.libertymutual.com/insurance-resources/life/how-does-life-insurance-work
Life insurance is a contract between an insurance company and policyholder. In exchange for a premium, the life insurance company agrees to pay a sum of money to one or more named beneficiaries upon the death of the policyholder. The purpose of life insurance is to help provide financial security to your loved ones upon your death. Some life policies also offer living benefits, meaning they can pay a part of the policy's death benefit while you're still alive if you're diagnosed with a covered illness that's considered chronic, critical, or terminal.

https://www.benefits.va.gov/insurance/
Life insurance is a contract wherein an insurance company provides a monetary benefit to a designated beneficiary upon the policyholder's death, in exchange for regular premium payments. In the context of the text, the following types of life insurance are highlighted:

1. **Veterans Affairs Life Insurance (VALife)** 

In [3]:
"""
Bundle up a set of reference material to send to the writer-citer model
"""
reference_material =  [
    {
        "link" : item['link'],
        "information" : item['choices'][0]['message']['content']
    }
    for item in website_refiner_responses
]

reference_material

[{'link': 'https://www.libertymutual.com/insurance-resources/life/how-does-life-insurance-work',
  'information': "Life insurance is a contract between an insurance company and policyholder. In exchange for a premium, the life insurance company agrees to pay a sum of money to one or more named beneficiaries upon the death of the policyholder. The purpose of life insurance is to help provide financial security to your loved ones upon your death. Some life policies also offer living benefits, meaning they can pay a part of the policy's death benefit while you're still alive if you're diagnosed with a covered illness that's considered chronic, critical, or terminal."},
 {'link': 'https://www.benefits.va.gov/insurance/',
  'information': "Life insurance is a contract wherein an insurance company provides a monetary benefit to a designated beneficiary upon the policyholder's death, in exchange for regular premium payments. In the context of the text, the following types of life insurance ar

In [4]:
# Generate a cited response with the writer-citer model
cited_response = writer_citer.generate_cited_response(
    rephrased_query=query,
    set_of_reference_query=reference_material
)


In [5]:
text = cited_response['choices'][0]['message']['content']
print(text)

Life insurance is a financial agreement between an insurance company and a policyholder, where the insurer promises to pay a designated sum of money to specified beneficiaries upon the policyholder's death, in exchange for premium payments[1]. The primary purpose of life insurance is to provide financial security to beneficiaries after the policyholder's demise[1][2].

There are various types of life insurance policies, including whole life insurance, which offers permanent coverage and may accumulate cash value over time; and term life insurance, which provides coverage for a specific period[1][4]. Some policies also include living benefits, which allow the policyholder to receive a portion of the death benefit if they are diagnosed with a qualifying illness[1].

Life insurance is crucial for different groups like veterans, who can access specialized insurance programs tailored to their needs, such as Veterans Affairs Life Insurance (VALife), which offers whole life coverage to servic

In [8]:
# Compute the visibility scores for the reference material
parsed_data = parse_citations_from_text(text)
scores = compute_citation_scores(parsed_data, num_citations=5, normalize=False)
normalized_scores = compute_citation_scores(parsed_data, num_citations=5, normalize=True)

In [9]:
scores

[62.22196765250655, 13.203387997277872, 0, 24.68394937224114, 9.56116058715572]

In [10]:
normalized_scores

[0.567353911619552,
 0.12039146477529429,
 0.0,
 0.2250738084782479,
 0.08718081512690587]